为了 TSNE 图能够正常显示中文

In [6]:
from matplotlib import rcParams
rcParams['font.family'] = 'Microsoft YaHei'

创建 session

In [7]:
sess = tf.Session()

读取字典

In [8]:
with open("/root/workspace/TestReviewBot/01_TestRviewBot_TensorFlow/02_Train_CBOW/embedding/temp/vocab.pkl", "rb") as f:
    id2word = pickle.load(f)
    id2word = [x.decode('utf-8') for x in id2word]
    word2id = dict(zip(id2word, np.arange(len(id2word))))
len(word2id)
word2id['大多']

15329

14065

把现有的 embeddings 模型加载进来， embeddings 变量的 shape 分别等于[词典中包含的单词数, 降维后的维数]

In [9]:
embeddings = tf.Variable(tf.random_uniform([len(word2id), 50]), name='embedding')
embedding_saver = tf.train.Saver({"w_in": embeddings})
embedding_saver.restore(sess, '/root/workspace/TestReviewBot/01_TestRviewBot_TensorFlow/02_Train_CBOW/embedding/temp/model.ckpt-70694191')

INFO:tensorflow:Restoring parameters from /root/workspace/TestReviewBot/01_TestRviewBot_TensorFlow/02_Train_CBOW/embedding/temp/model.ckpt-70694191


定义计算图，用于查找指定单词空间关系最近的词

In [10]:
nemb = tf.nn.l2_normalize(embeddings, 1)
nearby_word = tf.placeholder(dtype=tf.int32)
nearby_emb = tf.gather(nemb, nearby_word)
nearby_dist = tf.matmul(nearby_emb, nemb, transpose_b=True)
nearby_val, nearby_idx = tf.nn.top_k(nearby_dist, min(1000, len(word2id)))

将单词转换为ID，然后查找空间关系最近的词

In [11]:
def nearby(words, num=20):
    ids = np.array([word2id.get(x, 0) for x in words])
    vals, idx = sess.run([nearby_val, nearby_idx], {nearby_word: ids})
    for i in range(len(words)):
        print("\n%s\n=====================================" % (words[i]))
        for (neighbor, distance) in zip(idx[i, :num], vals[i, :num]):
            print("%-20s %6.4f" % (id2word[neighbor], distance))
#valid_words = ['多个', '大量', '常见', '长时间', '许多', '一些', '快速', '最低','最大','小时']
valid_words = ['多个']
nearby(valid_words)


多个
多个                   1.0000
一个                   0.7936
大量                   0.7908
某个                   0.7190
个                    0.7154
卡上                   0.7044
单个                   0.6949
其中                   0.6844
已有                   0.6577
某一                   0.6556
单独                   0.6491
若干                   0.6452
含有                   0.6452
某                    0.6380
中有                   0.6282
两个                   0.6269
卡中                   0.6212
同名                   0.6187
各种类型                 0.6053
任意                   0.6048


定义计算图，正则化 embedding, 用于计算每个单词在 embedding 中的具体值

In [148]:
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
normalized_embeddings = embeddings / norm
final_embeddings = sess.run(normalized_embeddings)

定义函数，将词向量绘制为2D图片，便于观察单词间的聚合关系

In [117]:
def plot_with_labels(low_dim_embs, labels):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.figure(figsize=(25, 25))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points', ha='right', va='bottom')
    plt.show()

创建 TSNE 对象, 具体作用没搞懂

In [112]:
from sklearn.manifold import TSNE
tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)

取出频率最高的 300 个单词的 embedding 值以及该 300 个单词的标签

In [ ]:
plot_only = 300
low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
labels = [id2word[i] for i in range(plot_only)]
plot_with_labels(low_dim_embs, labels)

发现一个神奇的问题，相同的输入，tsne.fit_transform转换出来的结果不一样 
http://blog.csdn.net/u012162613/article/details/45920827 http://www.datakit.cn/blog/2015/08/06/t_SNE.html http://bindog.github.io/blog/2016/06/04/from-sne-to-tsne-to-largevis/#0x02-从sne说起 在网上找了几篇帖子，原来 TSNE 是非线性降维, 总的来说也是个机器学习的过程, 突然发现 TSNE 是从 sklearn 中 import 的, 还能 fit, 好吧, 是我太蠢... 效果很好，但是每次就是不一样...

In [107]:
plot_only = 100

tmp1 = final_embeddings[:plot_only, :]
low_dim_embs1 = tsne.fit_transform(tmp1)
labels1 = [id2word[i] for i in range(plot_only)]

tmp2 = final_embeddings[:plot_only, :]
low_dim_embs2 = tsne.fit_transform(tmp2)
labels2 = [id2word[i] for i in range(plot_only)]

all((tmp1 == tmp2).flatten())
all((low_dim_embs1 == low_dim_embs2).flatten())
labels2 == labels2

True

False

True

创建目录

In [18]:
shutil.rmtree('log')
os.makedirs('log')

将 word 和 id 生成 tsv 文件

In [19]:
with open('log/metadata.tsv', 'w') as f:
    for word in id2word:
        _ = f.write(word + '\n')

创建一个 Tensorflow 的 Summary

In [20]:
summary_writer = tf.summary.FileWriter('log', sess.graph)
config = projector.ProjectorConfig()
embedding_conf = config.embeddings.add()
embedding_conf.tensor_name = 'embedding:0'
embedding_conf.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(summary_writer, config)

保存模型

In [21]:
saver = tf.train.Saver()
saver.save(sess, os.path.join('log', "model.ckpt"))

'log/model.ckpt'